In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [3]:
def NLM(w0, bu0, bs0, nt):
    w = np.zeros(nt)
    bu = np.zeros(nt)
    bs = np.zeros(nt)

    w[0] = w0
    bu[0] = bu0
    bs[0] = bs0
    for t in range(nt-1):
        if bs[t] > bu[t]:
            w[t+1] = w[t] + bs[t]*dt
        else:
            w[t+1] = w[t] + bu[t]*dt

        bu[t+1] = bu[t] - (Nu**2)*w[t]*dt
        bs[t+1] = bs[t] - (Ns**2)*w[t]*dt
    return w0, bu0, bs0

In [4]:
def TLM(w0, bu0, bs0, wp0, bup0, bsp0, nt):
    # basic state?
    w = np.zeros(nt)
    bu = np.zeros(nt)
    bs = np.zeros(nt)

    # p for prime, perturbation
    wp = np.zeros(nt)
    bup = np.zeros(nt)
    bsp = np.zeros(nt)

    # initial condition for basic state
    w[0] = w0
    bu[0] = bu0
    bs[0] = bs0

    # initial condition for perturbaitons are all zeros, 
    wp[0] = wp0
    bup[0] = bup0
    bsp[0] = bsp0
    # no need to specify
    for t in range(nt-1):
        if bs[t] > bu[t]:
            wp[t+1] = wp[t] + dt*bup[t]
            w[t+1] = w[t] + bs[t]*dt   
        else:
            wp[t+1] = wp[t] + dt*bsp[t]
            w[t+1] = w[t] + bu[t]*dt

        bup[t+1] = bup[t] - (Nu**2)*wp[t]*dt
        bu[t+1] = bu[t] - (Nu**2)*w[t]*dt
        bsp[t+1] = bsp[t] - (Ns**2)*wp[t]*dt
        bs[t+1] = bs[t] - (Ns**2)*w[t]*dt
        
    return wp, bup, bsp